In [1]:
# Personal 
from data.TrajectorySet import TrajectorySet
from data.Sampler import Sampler 
from data.DatasetCL import DatasetCL 
from models.cl_model import mlpCL
from trainer.cl_trainer import train_cl
from utils.visualizations import visualize_embeddings
from utils.tensor_utils import split_data

# Misc
import minari 
import os

# Torch 
import torch 

# PyTorch Lightning 
import pytorch_lightning
from pytorch_lightning.loggers import WandbLogger

In [2]:
MINARI_DATASET = minari.load_dataset("D4RL/pointmaze/large-v2")
PROJECT_ROOT = os.getcwd() 
CHECKPOINT_PATH = PROJECT_ROOT + "/saved_models"

PROJECT_NAME = "Contrastive Learning RL"
RUN_NAME = "best-model-gaussian-15-fullbatch"
FILENAME = "best_model_gaussian_15_fullbatch"

CONFIG = {
        "distribution": "g",
        "num_state_pairs": 4096,
        "k": 1,
        "lr": 1e-3,
        "weight_decay": 1e-5, 
        "temperature": 30,
        "max_epochs": 1000,
        "filename": FILENAME,
        "device": "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
    }

In [3]:
wandb_logger = WandbLogger(
        project=PROJECT_NAME, 
        name=RUN_NAME, 
        save_dir = PROJECT_ROOT, 
        log_model=True,
        config = CONFIG) 

dist = CONFIG["distribution"]
num_state_pairs = CONFIG["num_state_pairs"]
k = CONFIG["k"]
lr = CONFIG["lr"]
weight_decay = CONFIG["weight_decay"]
temperature = CONFIG["temperature"]
max_epochs = CONFIG["max_epochs"]
device = CONFIG["device"]
filename = CONFIG["filename"]

In [4]:
T = TrajectorySet(dataset=MINARI_DATASET)
S = Sampler(T, dist=dist, b=3, sigma=15)

ds = DatasetCL(S, num_state_pairs=num_state_pairs, k=k)
data = ds.get_batch() #change to list format to ensure correct split

#train, val = split_data(data=data, split_val=0.8)

train_dataset = DatasetCL(data=data, k = k)
#val_dataset = DatasetCL(data= val, k = k)

In [5]:
MINIBATCH_SIZE = len(train_dataset)
print("Minibatch size:", MINIBATCH_SIZE)
print(type(MINIBATCH_SIZE))

Minibatch size: 4096
<class 'int'>


In [6]:
model = train_cl(cl_model=mlpCL, 
                train_ds=train_dataset, 
                batch_size= MINIBATCH_SIZE,
                logger=wandb_logger, 
                checkpoint_path=CHECKPOINT_PATH,
                max_epochs=max_epochs,
                filename=filename,  
                device = device, 
                lr=lr, 
                temperature=temperature, 
                weight_decay = weight_decay)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 10


c:\Users\ray\AppData\Local\anaconda3\envs\CL_RL_Cuda\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: Currently logged in as: ray-s (ray-s-university-of-alberta) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


c:\Users\ray\AppData\Local\anaconda3\envs\CL_RL_Cuda\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Users\ray\Documents\2025 RA\contrastive-learning-RL\saved_models exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params | Mode 
--------------------------------------------
0 | mlp  | Sequential | 44.5 K | train
--------------------------------------------
44.5 K    Trainable params
0         Non-trainable params
44.5 K    Total params
0.178     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode
c:\Users\ray\AppData\Local\anaconda3\envs\CL_RL_Cuda\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.
c:\Users\ray\AppData\Local\anaconda3\envs\CL_RL_Cuda\lib\site-packages\pytorch_lightning\loops\fit_loop.py:310: 

Epoch 314:   0%|          | 0/1 [00:00<?, ?it/s, v_num=25a5, train/nll_loss_step=5.530, train/top1_step=0.0222, train/top5_step=0.0909, train/nll_loss_epoch=5.530, train/top1_epoch=0.0222, train/top5_epoch=0.0909]             


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined